In [1]:
import os
import re

import numpy as np
import pandas as pd

# os.chdir("/home/yujia/Project/2023-07-20-individual_MR/dat/02_base_data/")

In [2]:
TC = pd.read_csv("/mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/TC/download/without_UKB_TC_INV_AFR_HRC_1KGP3_others_ALL.meta.singlevar.results.gz", sep="\t", compression = 'gzip')

print("Total SNP in the GWAS data: ")
print(TC.shape)

Total SNP in the GWAS data: 
(31916286, 14)


In [3]:
TC

,rsID,CHROM,POS_b37,REF,ALT,N,N_studies,POOLED_ALT_AF,EFFECT_SIZE,SE,pvalue_neg_log10,pvalue,pvalue_neg_log10_GC,pvalue_GC
0,rs376342519,1,10616,C,CCGCCGTTGCAAAGGCGCGCCG,4215,2,0.02310,0.248136,0.174411,0.810171,0.1550,0.778475,0.167
1,rs575272151,1,11008,C,G,4215,2,0.04960,-0.015391,0.045145,0.134797,0.7330,0.130610,0.740
2,rs544419019,1,11012,C,G,4215,2,0.04960,-0.015391,0.045145,0.134797,0.7330,0.130610,0.740
3,rs62635286,1,13116,T,G,4215,2,0.01510,-0.115294,0.070058,1.000750,0.0998,0.960389,0.110
4,rs62028691,1,13118,A,G,4215,2,0.01510,-0.115294,0.070058,1.000750,0.0998,0.960389,0.110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31916281,rs8142977,22,51239304,C,T,4215,2,0.00617,0.080884,0.101568,0.370769,0.4260,0.357811,0.439
31916282,rs561893765,22,51239794,C,A,19513,9,0.03060,-0.000060,0.029855,0.000692,0.9980,0.000673,0.998
31916283,rs5770837,22,51240820,C,T,6468,4,0.10900,0.049207,0.031671,0.919894,0.1200,0.883238,0.131
31916284,rs7287738,22,51241285,T,G,7412,2,0.00459,-0.033741,0.108295,0.121839,0.7550,0.118089,0.762


In [4]:
# rename the data
TC.rename(columns={"CHROM":"CHR", "POS_b37":"BP", "rsID":"SNP", "REF":"A2", "ALT":"A1", "POOLED_ALT_AF":"EAF", "EFFECT_SIZE":"BETA", "SE":"SE", "pvalue":"P", "N":"N"}, inplace=True)

# convert EAF to MAF
TC['MAF'] = TC['EAF'].apply(lambda x : x if x <= 0.5 else 1-x)

# Uppercase the A1 and A2
TC['A1'] = TC['A1'].str.upper()
TC['A2'] = TC['A2'].str.upper()

In [5]:
TC

,SNP,CHR,BP,A2,A1,N,N_studies,EAF,BETA,SE,pvalue_neg_log10,P,pvalue_neg_log10_GC,pvalue_GC,MAF
0,rs376342519,1,10616,C,CCGCCGTTGCAAAGGCGCGCCG,4215,2,0.02310,0.248136,0.174411,0.810171,0.1550,0.778475,0.167,0.02310
1,rs575272151,1,11008,C,G,4215,2,0.04960,-0.015391,0.045145,0.134797,0.7330,0.130610,0.740,0.04960
2,rs544419019,1,11012,C,G,4215,2,0.04960,-0.015391,0.045145,0.134797,0.7330,0.130610,0.740,0.04960
3,rs62635286,1,13116,T,G,4215,2,0.01510,-0.115294,0.070058,1.000750,0.0998,0.960389,0.110,0.01510
4,rs62028691,1,13118,A,G,4215,2,0.01510,-0.115294,0.070058,1.000750,0.0998,0.960389,0.110,0.01510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31916281,rs8142977,22,51239304,C,T,4215,2,0.00617,0.080884,0.101568,0.370769,0.4260,0.357811,0.439,0.00617
31916282,rs561893765,22,51239794,C,A,19513,9,0.03060,-0.000060,0.029855,0.000692,0.9980,0.000673,0.998,0.03060
31916283,rs5770837,22,51240820,C,T,6468,4,0.10900,0.049207,0.031671,0.919894,0.1200,0.883238,0.131,0.10900
31916284,rs7287738,22,51241285,T,G,7412,2,0.00459,-0.033741,0.108295,0.121839,0.7550,0.118089,0.762,0.00459


In [6]:
# reorder the column
cols = ["CHR", "BP", "SNP", "A1", "A2", "N", "SE", "P", "BETA", "MAF", "EAF"]
TC = TC[cols]

# Remove NaN row
TC = TC.dropna()

# convert to correct column type
TC = TC.astype({'CHR':'int', 'BP':'int', "SNP":'str', 'A1':'str', 'A2':'str', 'N':'int', 'SE':'float', 'P':'float', 'BETA':'float', 'MAF':'float', 'EAF':'float'})

# remove SNP without SNPID
TC = TC[TC['SNP'].str.startswith('rs')]

# save the formatted results
TC.to_csv("/mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/TC/QC_AFR/TC_formated_nadrop.tsv.gz", sep="\t", index=False)

print("SNP left after basic QC: ")
print(TC.shape)

SNP left after basic QC: 
(31763752, 11)


In [7]:
TC

,CHR,BP,SNP,A1,A2,N,SE,P,BETA,MAF,EAF
0,1,10616,rs376342519,CCGCCGTTGCAAAGGCGCGCCG,C,4215,0.174411,0.1550,0.248136,0.02310,0.02310
1,1,11008,rs575272151,G,C,4215,0.045145,0.7330,-0.015391,0.04960,0.04960
2,1,11012,rs544419019,G,C,4215,0.045145,0.7330,-0.015391,0.04960,0.04960
3,1,13116,rs62635286,G,T,4215,0.070058,0.0998,-0.115294,0.01510,0.01510
4,1,13118,rs62028691,G,A,4215,0.070058,0.0998,-0.115294,0.01510,0.01510
...,...,...,...,...,...,...,...,...,...,...,...
31916281,22,51239304,rs8142977,T,C,4215,0.101568,0.4260,0.080884,0.00617,0.00617
31916282,22,51239794,rs561893765,A,C,19513,0.029855,0.9980,-0.000060,0.03060,0.03060
31916283,22,51240820,rs5770837,T,C,6468,0.031671,0.1200,0.049207,0.10900,0.10900
31916284,22,51241285,rs7287738,G,T,7412,0.108295,0.7550,-0.033741,0.00459,0.00459


In [8]:
# =============================
# Step 2: remove SNPs with MAF < 0.01
# =============================

cmd = "gunzip -c /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/TC/QC_AFR/TC_formated_nadrop.tsv.gz |" + \
      "awk 'NR==1 || ($10 > 0.01) {print}' |" + \
      "gzip > /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/TC/QC_AFR/TC.standardGWASQC.gz"
os.system(cmd)

cmd = "zcat /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/TC/QC_AFR/TC.standardGWASQC.gz | wc -l"
print("SNP left after removing SNPs with MAF < 0.01: ")
print(os.popen(cmd).read())

SNP left after removing SNPs with MAF < 0.01: 
16298613



In [9]:
# =============================
# Step 3: remove duplicate SNPS
# =============================

cmd = "gunzip -c /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/TC/QC_AFR/TC.standardGWASQC.gz |" + \
      "awk '{seen[$3]++; if(seen[$3]==1){ print}}' |" + \
      "gzip - > /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/TC/QC_AFR/TC.nodup.gz"
os.system(cmd)

cmd = "zcat /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/TC/QC_AFR/TC.nodup.gz | wc -l"
print("SNP left after removing duplicate SNPs: ")
print(os.popen(cmd).read())

SNP left after removing duplicate SNPs: 
16242897



In [10]:
# =============================
# Step 4: keep non-ambiguous SNPS
# =============================

cmd = "gunzip -c /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/TC/QC_AFR/TC.nodup.gz | " + \
      "awk '!( ($4==\"A\" && $5==\"T\") || ($4==\"T\" && $5==\"A\") || ($4==\"G\" && $5==\"C\") || ($4==\"C\" && $5==\"G\")) {print}' | " + \
      "gzip > /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/TC/QC_AFR/TC.QC.gz"
os.system(cmd)

cmd = "zcat /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/TC/QC_AFR/TC.QC.gz | wc -l"
print("SNP left after removing ambiguous SNPs: ")
print(os.popen(cmd).read())

SNP left after removing ambiguous SNPs: 
13943507

